# Notebook 2 : Sélection et évaluation de modèles

Notebook préparé par [Chloé-Agathe Azencott](http://cazencott.info) avec l'aide d'[Arthur Imbert](https://github.com/Henley13) et de [Giann Karlo](https://www.giannkarlo.info/).

Dans ce notebook il s'agit
* d'évaluer un modèle sur un jeu de test
* de choisir la valeur d'un hyperparamètre d'un algorithme d'apprentissage
* de comprendre l'intérêt de la régression polynomiale et de la régularisation

In [ ]:
# load numpy as np, matplotlib as plt
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
plt.rc('font', **{'size': 12}) # règle la taille de police globalement pour les plots (en pt)

In [ ]:
import pandas as pd

## 1. Chargement des données

#### Ormeau
Déterminer l'âge des ormeaux à partir de mesures physiques.  L'âge des ormeaux est déterminé en coupant la coquille à travers le cône, en la colorant et en comptant le nombre d'anneaux à l'aide d'un microscope, une tâche laborieuse et de longue durée.  D'autres mesures, plus faciles à obtenir, sont utilisées pour déterminer l'âge. Notre but est donc prédire l'âge des ormeaux de manière automatique et aider les biologistes à déterminer l'âge des ormeaux dans un endroit donné.

#### *Vin (optionnel)*
Nous allons travailler avec un jeu de données contenant des informations physico-chimiques sur un certain nombre de vins portugais (vinho verde), ainsi que les notes qui leur ont été attribuées par des gens qui les ont goûtés. Notre but est d'automatiser ce processus : nous voulons prédire directement la note des vins à partir de leurs caractéristiques physico-chimiques, afin d'assister les œnologues, améliorer la production de vin, et cibler les goûts de consomateurs de niche.

-----

Les deux jeux de données sont disponibles sur l'archive UCI de jeux de données de machine learning, sur laquelle vous trouverez de nombreux jeux de données classiques : [wine-quality](http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/) et [abalone](https://archive.ics.uci.edu/dataset/1/abalone). Pas besoin de le télécharger, ils sont déjà dans votre répertoire, dans le fichier `data/abalone.csv` ou `data/winequality-white.csv`. Nous allons le charger avec pandas :

In [ ]:
df = pd.read_csv('data/abalone.csv', # nom du fichier
                   sep="," # séparateur de colonnes
                   )

# df = pd.read_csv('data/winequality-white.csv', # nom du fichier
#                    sep="," # séparateur de colonnes
#                    )

__Alternativement :__ Si vous avez besoin de télécharger le fichier (par exemple sur colab) :

In [ ]:
#!wget https://raw.githubusercontent.com/chagaz/cp-ia-intro-ml-2022/main/2-Selection/data/winequality-white.csv
!wget https://raw.githubusercontent.com/ThomasWalter/CourseFoundationsML/SIA2025/Notebooks/2-Selection/data/abalone.csv

# df = pd.read_csv('winequality-white.csv', # nom du fichier
#                    sep=";" # séparateur de colonnes
#                    )

df = pd.read_csv('abalone.csv', # nom du fichier
                   sep="," # séparateur de colonnes
                   )

Nous pouvons maintenant examiner ce fichier directement dans notre notebook, par exemple en en regardant les premières lignes :

In [ ]:
df.head()

### Transformation one-hot

In [ ]:
# Créer un nouveau data frame où la colomne 'sex' est remplacée par son encodage 'one-hot'
df_dummies = pd.get_dummies(df, columns=['sex'])

In [ ]:
df_dummies.head()

### Création des matrices X et y de données

In [ ]:
X = np.array(df_dummies.drop(columns=['rings']))

In [ ]:
y = np.array(df['rings'])

In [ ]:
print(X.shape, y.shape)

__Question :__ Combien d'exemples d'apprentissage contiennent les données ? Combien de variables ?

__Question :__ Que pensez-vous de l'utilisation d'une régression linéaire pour résoudre ce problème ?

### Transformation en un problème de classification binaire

Nous allons essayer de classifier les anciens et les plus jeunes ormeaux (score >= 10).

In [ ]:
y = np.where(y >= 10, 1, 0)

## 2. Séparation des données en un jeu d'entraînement et un jeu de test

Pour pouvoir évaluer un modèle d'apprentissage de façon non-biaisée, nous avons besoin de créer un jeu de test contenant des données sur lequel le modèle n'a pas été entraîné. Ce jeu de test correspond à des données « nouvelles ».

Pour ce faire, nous allons utiliser la fonction [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) du module `model_selection` de scikit-learn :

In [ ]:
from sklearn import model_selection

X_train, X_test, y_train, y_test = \
    model_selection.train_test_split(X, y,
                                    test_size=0.3, # 30% des données dans le jeu de test
                                    random_state=42 # graine du générateur aléatoire
                                    )

Fixer la graine du générateur aléatoire nous permet d'obtenir les mêmes jeux d'entraînement et de test en relançant la commande.

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

__Question :__ Combien d'échantillons le jeu d'entraînement (X_train, y_train) contient-il ? Et le jeu de test (X_test, y_test) ?

### Transformation des variables
Nous avons vu dans le Notebook 1 qu'il est plus raisonnable de centrer-réduire les variables avant de procéder.

N'oublions pas que le jeu de test est prétendu non-connu au moment de l'entraînement : il faut utiliser __uniquement le jeu d'entraînement__ pour centrer-réduire les données.

In [ ]:
from sklearn import preprocessing

In [ ]:
# Créer un "standardiseur" et le calibrer sur les données d'entraînement
std_scaler = preprocessing.StandardScaler().fit(X_train)

# Appliquer la standardisation aux données d'entraînement
X_train_scaled = std_scaler.transform(X_train)

# Appliquer la standardisation aux données de test
X_test_scaled = std_scaler.transform(X_test)

## 3. Plus proches voisins

Nous allons maintenant évaluer la capacité d'un algorithme des plus proches voisins à classifier les ormeaux.

Pour cela, nous faisons appel à la classe [KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) du module `neighbors` de scikit-learn.

In [ ]:
from sklearn import neighbors

### Entrainement sur le jeu d'entrainement

Comme dans le Notebook 1, on commence par instancier un objet de la classe qui nous intéresse :

In [ ]:
model_knn = neighbors.KNeighborsClassifier()

On peut ensuite l'entraîner sur les données d'entrainement centrées-réduites :

In [ ]:
model_knn.fit(X_train_scaled, y_train)

### Prédictions sur le jeu de test

On peut maintenant utiliser le classifieur entraîné sur les données de test, toujours centrées-réduites :

In [ ]:
y_pred_knn = model_knn.predict(X_test_scaled)

### Évaluation de la performance

De nombreuses métriques permettent d'évaluer la performance d'un algorithme de classification (voir [la doc de scikit-learn à ce sujet](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics)). La __matrice de confusion__ en particulier permet de visualiser combien d'exemple de chaque classe reçoivent chacune des étiquettes :

In [ ]:
from sklearn import metrics

In [ ]:
metrics.ConfusionMatrixDisplay.from_predictions(y_test, y_pred_knn)

La matrice de confusion peut être résumée par le [score F1](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score).

Le **score F1** est utile pour évaluer les performances d'un modèle de classification. Il combine deux autres mesures importantes : la **precision** et le **recall**. Le score F1 est la moyenne harmonique de la precision et du recall, fournissant un score unique qui équilibre les deux.

Le score F1 est utile lorsque vous disposez d'un ensemble de données déséquilibré, par exemple un ensemble de données sur la **désinformation** où 99 % des publications ne sont pas fausses et seulement 1 % le sont. Un modèle qui prédit simplement « non faux » à chaque fois serait précis à 99 %, mais il serait inutile car il ne trouverait jamais les fausses informations. Le score F1 fournit une bien meilleure évaluation dans ces cas-là.

In [ ]:
print("F1 du kNN sur le jeu de test : %.3f" % metrics.f1_score(y_test, y_pred_knn))

## 4. Sélection du nombre de plus proches voisins

__Question :__ Combien de plus proches voisins a-t-on utilisé dans la section précédente ? Appuyez-vous sur la documentation, par exemple en tapant
```
 help(neighbors.KNeighborsClassifier)
 ```
dans une cellule ci-dessous.

In [ ]:
help(neighbors.KNeighborsClassifier)

### Mise en place d'une validation croisée

Le nombre de plus proches voisins (`n_neighbors`) est un __hyperparamètre__ de l'algorithme des plus proches voisins : il ne fait pas partie des paramètres du modèle appris par l'algorithme, mais nous devons le fixer nous-mêmes avant l'entraînement.

Nous allons maintenant _choisir_ ce nombre de plus proches voisins par une procédure de __recherche en grille__ (_gridsearch_), qui consiste à _comparer_ les performances de modèles entraînés en utilisant des valeurs prédéfinies (la grille) de l'hyperparamètre.

Attention ! Si nous voulons pouvoir utiliser le jeu de test pour évaluer l'erreur de généralisation du modèle utilisant la valeur optimale du nombre de plus proches voisins, nous ne pouvons pas l'utiliser aussi pour cette étape de sélection, car sinon nous pourrions biaiser le modèle et surapprendre.

Pour comparer nos modèles __sur le jeu d'entraînement__, nous allons utiliser une __validation croisée__, encore une fois grâce au module [model-selection](http://scikit-learn.org/stable/model_selection.html#model-selection) de scikit-learn.

In [ ]:
n_folds = 10

# Créer un objet KFold qui permettra de cross-valider en n_folds folds
kf = model_selection.KFold(n_splits=n_folds,
                           shuffle=True, # mélanger les échantillons avant de créer les folds
                           random_state=42 # graine pour le mélange des échantillons et pour la reproductibilité
                          )

# Utiliser kf pour partager le jeu d'entraînement en n_folds folds.
# kf.split retourne un iterateur (consommé après une boucle).
# Pour pouvoir se servir plusieurs fois des mêmes folds, nous transformons cet itérateur en liste d'indices :
kf_indices = list(kf.split(X_train))

`kf_indices` contient 10 paires de deux vecteurs d'indices.

Chacune de ces paires correspond à un fold.

Le premier vecteur donne les indices des échantillons formant la partie entraînement de ce fold. Le deuxième donne les indices des échantillons formant la partie test de ce fold.

In [ ]:
for (idx, fold) in enumerate(kf_indices):
    print("Le fold %d contient %d observations pour l'entraînement and %d observations pour le test" % (idx, len(fold[0]), len(fold[1])))

__Question :__ Combien de fois chaque échantillon apparaît-il dans la partie entraînement d'un fold ? Dans la partie test ? (Il n'est pas nécessaire d'écrire de code pour répondre.)

### Recherche en grille

Nous allons commencer par définir une __grille__ de valeurs d'hyperparamètres, c'est à dire une liste de valeurs du nombre de plus proches voisins à évaluer :

In [ ]:
k_values = np.arange(3, 50, step=2)

In [ ]:
k_values

__Question :__ Pourquoi sélectionner uniquement des nombres impairs de voisins ?

Nous allons maintenant utiliser la classe [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) du module `model_selection` de scikit-learn pour déterminer la valeur optimale du nombre de plus proches voisins par recherche en grille :

In [ ]:
# Instanciation d'un objet GridSearchCV
grid = model_selection.GridSearchCV(neighbors.KNeighborsClassifier(), # prédicteur à évaluer
                                    {'n_neighbors':k_values}, # dictionnaire de valeurs d'hyperparamètres
                                    cv=kf_indices, # validation croisée à utiliser
                                    scoring='f1' # métrique d'évaluation de la performance
                                   )

Nous allons aussi utiliser la [commande magique time](https://ipython.readthedocs.io/en/stable/interactive/magics.html) pour mesurer le temps de calcul d'une cellule de notre notebook.

In [ ]:
%%time
# Utilisation de cet objet sur les données d'entraînement (centrées-réduites)
grid.fit(X_train_scaled, y_train)

La valeur optimale de l'hyperparamètre est donnée par :

In [ ]:
print(grid.best_params_)

Le code suivant permet d'afficher la performance du modèle selon la valeur de l'hyperparamètre :

In [ ]:
mean_test_score = grid.cv_results_['mean_test_score']
stde_test_score = grid.cv_results_['std_test_score'] / np.sqrt(n_folds) # standard error

p = plt.plot(k_values, mean_test_score)
plt.plot(k_values, (mean_test_score + stde_test_score), '--', color=p[0].get_color())
plt.plot(k_values, (mean_test_score - stde_test_score), '--', color=p[0].get_color())
plt.fill_between(k_values, (mean_test_score + stde_test_score),
                 (mean_test_score - stde_test_score), alpha=0.2)

best_index = np.where(k_values == grid.best_params_['n_neighbors'])[0][0]
plt.scatter(k_values[best_index], mean_test_score[best_index])


plt.xlabel('nombre de plus proches voisins')
plt.ylabel('F1')
plt.title("Performance (en validation croisée) le long de la grille")
plt.show()

### Modèle optimal de plus proches voisins

In [ ]:
print("Meilleur F1 en validation croisée : %.3f" % grid.best_score_)

Le modèle entraîné sur l'intégralité des données fournies à `grid.fit` avec la (les) meilleure(s) valeur(s) d'hyperparamètre(s) est donné par `grid.best_estimator_`.

In [ ]:
y_pred_knn_opt = grid.best_estimator_.predict(X_test_scaled)

In [ ]:
metrics.ConfusionMatrixDisplay.from_predictions(y_test, y_pred_knn_opt)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred_knn_opt))

Version alternative (ancienne version de scikit-learn) :

In [ ]:
print("F1 du kNN (k optimal) sur le jeu de test : %.3f" % metrics.f1_score(y_test, y_pred_knn_opt))

## 5. Régression logistique régularisée

### Performance d'une régression logistique non-régularisée

Nous allons maintenant entraîner une régression __logistique__ (car nous avons un problème de classification) _sur le jeu d'entraînement_ et l'évaluer _sur le jeu de test_.

Nous utilisons la classe [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) du module `linear_model` de scikit-learn.

In [ ]:
from sklearn import linear_model

In [ ]:
# Créer un modèle de régression linéaire
model_rlog = linear_model.LogisticRegression(penalty=None # modèle non régularisé pour l'instant
                                            )

# Entraîner ce modèle sur (X_train_scaled, y_train)
model_rlog.fit(X_train_scaled, y_train)

In [ ]:
# Prédire les étiquettes du jeu de test
y_pred_rlog = model_rlog.predict(X_test_scaled)

In [ ]:
metrics.ConfusionMatrixDisplay.from_predictions(y_test, y_pred_rlog)

In [ ]:
print("F1 d'une régression logistique sur le jeu de test : %.3f" % metrics.f1_score(y_test, y_pred_rlog))

__Question :__ Que pensez-vous de la qualité du modèle ?

### Coefficients du modèle

In [ ]:
# Calculer le nombre de variables
num_features = X_train.shape[1]

# Afficher pour chaque variable son coefficient dans le modèle
plt.scatter(range(num_features), # en abcisse : indices des variables
            model_rlog.coef_ # en ordonnées : leur poids dans le modèle
           )

# Étiqueter les graduations de l'axe des abcsisses
tmp = plt.xticks(range(num_features), # une marque par variable
                 list(df_dummies.drop(columns=['rings']).columns),  # afficher le nom de la variable
                 rotation=90, # tourner les étiquettes de 90 degrés
                 fontsize=14)

# Étiqueter les axes
tmp = plt.xlabel('Variable', fontsize=14)
tmp = plt.ylabel('Coefficient', fontsize=14)
plt.show()

In [ ]:
# Afficher les coefficients avec leurs noms de variables correspondants
for i, col_name in enumerate(df_dummies.drop(columns=['rings']).columns):
    print(f"{col_name}: {model_rlog.coef_[0][i]:.4f}")

### Régularisation ridge

Nous allons maintenant ajouter une régularisation l2 (ou ridge) à cette régression logistique.

Ici il y a peu de variables et leurs coefficients prennent des valeurs faibles : il n'est pas certain que la régularisation soit nécessaire, mais comme ce jeu de données comporte peu de variables, nous pouvons l'utiliser pour regarder l'effet de la régularisation sur les valeurs des coefficients du modèle appris.

Commençons par nous donner une grille de valeurs pour le paramètre de régularisation `C`.

Attention ! Plus `C` est grand, _moins_ il y a de régularisation.

In [ ]:
c_values = np.logspace(-6, 6, 50)
c_values

Nous allons maintenant utiliser non pas `GridSearchCV` mais implémenter notre recherche en grille nous-mêmes, afin d'avoir accès aux valeurs des coefficients de chacun des modèles :

In [ ]:
%%time

f1_per_c = [] # pour enregistrer les valeurs du score F1 pour chacune des 50 valeurs de C
weights_per_c = [] # pour enregistrer les coefficients associés à chaque variable,
                   # pour les 50 valeurs de C
for c_val in c_values:
    # Créer un modèle de régression logistique régularisée par le paramètre c_val
    model_ridge = linear_model.LogisticRegression(penalty='l2', C=c_val)

    # Calculer la performance en validation croisée du modèle
    f1 = model_selection.cross_val_score(model_ridge, # prédicteur à évaluer
                                         X_train_scaled, y_train, # données d'entrainement
                                         cv=kf_indices, # validation croisée à utiliser
                                         scoring='f1' # métrique d'évaluation de la performance
                                         )
    f1_per_c.append(f1)

    # Entrainer le modèle sur le jeu d'entrainement total
    model_ridge.fit(X_train_scaled, y_train)

    # Enregistrer les coefficients de régression
    weights_per_c.append(model_ridge.coef_[0])

### Évolution de la performance en fonction du coefficient de régularisation

In [ ]:
mean_test_score = np.mean(np.array(f1_per_c), axis=1)
stde_test_score = np.std(np.array(f1_per_c), axis=1) / np.sqrt(n_folds) # standard error

p = plt.plot(c_values, mean_test_score)
plt.plot(c_values, (mean_test_score + stde_test_score), '--',
         color=p[0].get_color()) # réutiliser la même couleur que précédemment au lieu d'avancer
plt.plot(c_values, (mean_test_score - stde_test_score), '--', color=p[0].get_color())
plt.fill_between(c_values, (mean_test_score + stde_test_score),
                 (mean_test_score - stde_test_score), alpha=0.2)


plt.xscale('log') # utiliser une échelle logarithmique en abcisse

# Étiqueter les axes
tmp = plt.xlabel('Valeur de C', fontsize=14)
tmp = plt.ylabel('F1 moyen', fontsize=14)

# Titre
tmp = plt.title("Performance (validation croisée) de la régression logistique", fontsize=14)
plt.show()

__Question :__ Comment l'erreur du modèle (en validation croisée) évolue-t-elle en fonction de la quantité de régularisation ?

### Modèle optimal de régression ridge

Nous pouvons maintenant sélectionner, parmi nos 50 modèles de régression ridge, celui qui a la plus petite erreur en validation croisée :

In [ ]:
# Trouver l'index de la valeur optimale de C
best_C_idx = np.argmax(np.mean(f1_per_c, axis=1))

# Valeur de C optimale
c_opt = c_values[best_C_idx]
print("Valeur de C optimale (regression ridge) : %.3e" % c_opt)

# MSE correspondante
print("Score F1 (validation croisée) du modèle de regression logistique régularisée optimal : %.2f +/- %.2f" % \
     (np.mean(np.array(f1_per_c)[best_C_idx]), # valeur moyenne
      np.std(np.array(f1_per_c)[best_C_idx]) # écart-type
     ))

### Évolution des coefficients de régression en fonction de la régularisation

In [ ]:
# Créer une figure
fig = plt.figure(figsize=(8, 5))

# Changer les couleurs pour une meilleure visualisation
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'purple', 'pink', 'brown', 'orange', 'teal', 'coral', 'lightblue', 'lime', 'lavender', 'turquoise', 'darkgreen', 'tan', 'salmon', 'gold'])

lines = plt.plot(c_values,
                 weights_per_c # ordonnée = valeurs des coefficients de régression
                )
plt.xscale('log') # échelle logarithmique en abcisse

# Afficher de nouveau (à l'abscisse 2x1e6) les coefficients de régression obtenus sans régularisation
for coeff in model_rlog.coef_[0]:
    plt.scatter([2e6], [coeff])

# Marquer la valeur optimale de C d'une barre verticale
plt.plot([c_opt, c_opt], [-5, 5], 'k--')

# Afficher la légende
tmp = plt.legend(lines, # récupérer l'identifiant
                 list(df.columns), # nom de chaque variable
                 frameon=False, # pas de cadre autour de la légende
                 loc=(1, 0),  # placer la légende à droite de l'image
                 fontsize=14)

tmp = plt.xlabel('Valeur de C', fontsize=14)
tmp = plt.ylabel('Coefficient de régression', fontsize=14)

tmp = plt.title('Régression logistique', fontsize=16)
plt.show()

In [ ]:
# Obtenir les coefficients du meilleur estimateur à partir de la recherche par grille
optimal_coefficients = weights_per_c[best_C_idx]

# Afficher les coefficients avec leurs noms de variables correspondants
for i, col_name in enumerate(df_dummies.drop(columns=['rings']).columns):
    print(f"{col_name}: {optimal_coefficients[i]:.4f} ")

__Question :__ Comment les coefficients du modèle évoluent-ils en fonction de la quantité de régularisation ?

__Question :__ Ces coefficients vous semblent-ils cohérents avec ceux obtenus pour la régression logistique non-régularisée ?

## 6. Régularisation ridge sur un cas d'école

Pour mieux comprendre la régularisation ridge, nous allons simuler un jeu de données non-linéaire qui prendra la forme d'une courbe sinusoïdale.

### Simulation de données

In [ ]:
nb_samples = 30

np.random.seed(13)

# vrai modèle
def true_model(X):
    return np.cos(1.5 * np.pi * X) * 5

# échantillons "ground truth" tirés du vrai modèle
X_ground_truth = np.linspace(0, 1, 100).reshape(-1, 1)
y_ground_truth = true_model(X_ground_truth)

# données = observations tirées du vrai modèle puis bruitées
X = np.sort(np.random.rand(nb_samples, 1))
y = true_model(X)
# ajout du bruit
y += np.random.randn(nb_samples, 1) * 0.3

print(X.shape, y.shape)

In [ ]:
# Dessiner le vrai modèle
plt.plot(X_ground_truth, y_ground_truth, label="Vrai modèle", linewidth=2)

# Afficher les données simulées
plt.scatter(X, y, label="Données simulées", marker="o")

plt.xlabel("X")
plt.ylabel("y")
plt.legend(loc="best")
plt.tight_layout()
plt.show()

### Séparation entrainement / test

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

### Régression linéaire

__Question :__ Combien de variables avons-nous dans notre problème ?

Entrainons une régression linéaire « classique » (comme celle vue dans le Notebook 1) sur `(X_train, y_train)` et évaluons sa performance d'une part sur le jeu d'entraînement et d'autre part sur le jeu de test.

__Question :__ Pourquoi comparer ces deux performances ?

In [ ]:
# Entrainement
reg = linear_model.LinearRegression()
# Entraîner le modèle linéaire
reg.fit(X_train, y_train)

#### Performance

In [ ]:
# RMSE
print("RMSE d'une régression linéaire :")
# Sur le jeu d'entrainement
rmse_reg_train = metrics.root_mean_squared_error(y_train, reg.predict(X_train))
print("\r train: {0:0.2f}".format(rmse_reg_train))
# Sur le jeu de test
rmse_reg_test = metrics.root_mean_squared_error(y_test, reg.predict(X_test))
print("\r test: {0:0.2f}".format(rmse_reg_test))

Nous allons maintenant afficher le modèle appris sur le graphe précédent.

In [ ]:
# Dessiner le vrai modèle
plt.plot(X_ground_truth, y_ground_truth, label="Vrai modèle", linewidth=2)

# Afficher le modèle appris
y_model = reg.predict(X_ground_truth)
plt.plot(X_ground_truth, y_model, label="Modèle appris", linewidth=2)

# Afficher les données simulées
plt.scatter(X_train, y_train, label="Données simulées (train)", marker="o")
plt.scatter(X_test, y_test, label="Données simulées (test)", marker="D")

# format plot
plt.xlabel("X")
plt.ylabel("y")
plt.title("Régression linéaire")
plt.legend(loc="best")
plt.tight_layout()
plt.show()

__Question :__ Que pensez-vous de la performance de la régression linéaire ici ?

### Régression polynomiale

La régression polynomiale consiste à apprendre un modèle non-linéaire en apprenant un modèle linéaire sur un nouvel ensemble de variables, formé de monomes des variables décrivant nos données.

De manière générale, pour un problème décrit par $p$ variables $(X_1, X_2, \dots, X_p)$, une régression polynomiale de degré $d$ est une régression linéaire sur les variables $(X_1, X_2, \dots, X_p, X_1^2, X_2^2 X_3^2, \dots, X_p^2, \dots, X_p^d)$. Remarquez que nous créons ainsi un grand nombre de variables, corrélées entre elles ; nous gagnons en finesse de modélisation, mais perdons en complexité du modèle, risque de surapprentissage, et fléau de la dimension.

Une telle transformation est possible avec la classe `PolynomialFeatures` de `sklearn.preprocessing`.

Ici, il s'agit donc de régresser une droite à partir des puissances de $X$ et non plus de $X$ uniquement : on approche le vrai modèle par un polynôme.

In [ ]:
# calcul des puissances de x, jusqu'au degré 15
polynomial_features = preprocessing.PolynomialFeatures(degree=15)#, include_bias=False)

# création des jeux de données correspondants
X_train_poly = polynomial_features.fit_transform(X_train)
X_test_poly = polynomial_features.transform(X_test)
X_ground_truth_poly = polynomial_features.transform(X_ground_truth)

print(X_train_poly.shape)
print(X_test_poly.shape)
print(X_ground_truth_poly.shape)

__Question :__ Combien de variables avons-nous maintenant ?

In [ ]:
# Entrainement
reg_poly = linear_model.LinearRegression()
reg_poly.fit(X_train_poly, y_train)

#### Performance

In [ ]:
# RMSE
print("RMSE d'une régression polynomiale :")
# Sur le jeu d'entrainement
rmse_reg_poly_train = metrics.root_mean_squared_error(y_train, reg_poly.predict(X_train_poly))
print("\r train: {0:0.2f}".format(rmse_reg_poly_train))
# Sur le jeu de test
rmse_reg_poly_test = metrics.root_mean_squared_error(y_test, reg_poly.predict(X_test_poly))
print("\r test: {0:0.2f}".format(rmse_reg_poly_test))

__Question :__ Comparez les performances du modèle sur le jeu d'entrainement et le jeu de test. Que conclure ?

Nous allons maintenant afficher le modèle appris sur le graphe précédent.

In [ ]:
# Dessiner le vrai modèle
plt.plot(X_ground_truth, y_ground_truth, label="Vrai modèle", linewidth=2)

# Afficher le modèle appris
plt.plot(X_ground_truth, reg_poly.predict(X_ground_truth_poly), label="Modèle appris", linewidth=2)

# Afficher les données simulées
plt.scatter(X_train, y_train, label="Données simulées (train)", marker="o")
plt.scatter(X_test, y_test, label="Données simulées (test)", marker="D")

# format plot
plt.xlabel("X")
plt.ylabel("y")
plt.title("Régression polynomiale")
plt.legend(loc="best")
plt.tight_layout()
plt.ylim([-6, 6])
plt.show()

__Question :__ Que pouvez-vous conclure sur le choix de la régression polynomiale ?

#### Coefficients du modèle

In [ ]:
# Calculer le nombre de variables
num_features = X_train_poly.shape[1]

# Afficher pour chaque variable son coefficient dans le modèle
plt.scatter(range(num_features), # en abcisse : indices des variables
            reg_poly.coef_ # en ordonnées : leur poids dans le modèle
           )

# Étiqueter les axes
tmp = plt.xlabel('Variable', fontsize=14)
tmp = plt.ylabel('Coefficient', fontsize=14)
plt.yscale("log")
plt.show()

__Question :__ Que remarquez-vous ? Faites bien attention à l'échelle des coefficients.

### Régression polynomiale régularisée ridge

Comme la régression polynomiale surapprend, nous allons maintenant lui appliquer un terme de régularisation ridge pour essayer de compenser cet effet.

In [ ]:
# Entrainement
ridge_poly = linear_model.Ridge(alpha=0.01, random_state=13)
ridge_poly.fit(X_train_poly, y_train)

#### Performance

In [ ]:
# RMSE
print("RMSE d'une régression polynomiale régularisée :")
# Sur le jeu d'entrainement
rmse_ridge_poly_train = metrics.root_mean_squared_error(y_train, ridge_poly.predict(X_train_poly))
print("\r train: {0:0.2f}".format(rmse_ridge_poly_train))
# Sur le jeu de test
rmse_ridge_poly_test = metrics.root_mean_squared_error(y_test, ridge_poly.predict(X_test_poly))
print("\r test: {0:0.2f}".format(rmse_ridge_poly_test))

__Question :__ Comparez les performances du modèle sur le jeu d'entrainement et le jeu de test. Que conclure ?

Nous allons maintenant afficher le modèle appris sur le graphe précédent.

In [ ]:
# Dessiner le vrai modèle
plt.plot(X_ground_truth, y_ground_truth, label="Vrai modèle", linewidth=2)

# Afficher le modèle appris
plt.plot(X_ground_truth, ridge_poly.predict(X_ground_truth_poly), label="Modèle appris", linewidth=2)

# Afficher les données simulées
plt.scatter(X_train, y_train, label="Données simulées (train)", marker="o")
plt.scatter(X_test, y_test, label="Données simulées (test)", marker="D")

# format plot
plt.xlabel("X")
plt.ylabel("y")
plt.title("Régression polynomiale régularisée")
plt.legend(loc="best")
plt.tight_layout()
plt.show()

__Question :__ Que pouvez-vous conclure sur le choix de la régularisation Ridge ?

#### Coefficients du modèle

In [ ]:
# Calculer le nombre de variables
num_features = X_train_poly.shape[1]

# Afficher pour chaque variable son coefficient dans le modèle
plt.scatter(range(num_features), # en abcisse : indices des variables
            ridge_poly.coef_ # en ordonnées : leur poids dans le modèle
           )

# Étiqueter les axes
tmp = plt.xlabel('Variable', fontsize=14)
tmp = plt.ylabel('Coefficient', fontsize=14)
plt.show()

__Question :__ Que remarquez-vous maintenant ? Quel est l'effet de la régularisation sur les coefficients du modèle ?

### Régression polynomiale régularisée lasso

In [ ]:
# Entrainement
lasso_poly = linear_model.Lasso(alpha=0.01, random_state=13)
lasso_poly.fit(X_train_poly, y_train)

#### Performance

In [ ]:
# RMSE
print("RMSE d'une régression polynomiale régularisée :")
# Sur le jeu d'entrainement
rmse_lasso_poly_train = metrics.root_mean_squared_error(y_train, lasso_poly.predict(X_train_poly))
print("\r train: {0:0.2f}".format(rmse_lasso_poly_train))
# Sur le jeu de test
rmse_lasso_poly_test = metrics.root_mean_squared_error(y_test, lasso_poly.predict(X_test_poly))
print("\r test: {0:0.2f}".format(rmse_lasso_poly_test))

Nous allons maintenant afficher le modèle appris sur le graphe précédent.

In [ ]:
# Dessiner le vrai modèle
plt.plot(X_ground_truth, y_ground_truth, label="Vrai modèle", linewidth=2)

# Afficher le modèle appris
plt.plot(X_ground_truth, lasso_poly.predict(X_ground_truth_poly), label="Modèle appris", linewidth=2)

# Afficher les données simulées
plt.scatter(X_train, y_train, label="Données simulées (train)", marker="o")
plt.scatter(X_test, y_test, label="Données simulées (test)", marker="D")

# format plot
plt.xlabel("X")
plt.ylabel("y")
plt.title("Régression polynomiale régularisée l1")
plt.legend(loc="best")
plt.tight_layout()
plt.show()

#### Coefficients du modèle

In [ ]:
# Calculer le nombre de variables
num_features = X_train_poly.shape[1]

# Afficher pour chaque variable son coefficient dans le modèle
plt.scatter(range(num_features), # en abcisse : indices des variables
            lasso_poly.coef_ # en ordonnées : leur poids dans le modèle
           )

# Étiqueter les axes
tmp = plt.xlabel('Variable', fontsize=14)
tmp = plt.ylabel('Coefficient', fontsize=14)
plt.show()

## Conclusion
Nous sommes arrivés à la fin de ce notebook. Voici un résumé de ce que nous avons couvert, avec les points clés :
- Nous avons utilisé la bibliothèque `scikit-learn` pour classer la l'ancienneté des ormeaux (ou la qualité du vin) à partir de variables continues (caractéristiques du ormeau ou du vin).
- Nous avons essayé un premier modèle de classificateur : [KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) (après avoir mis à l'échelle les données).
- Nous avons évalué la performance du modèle à l'aide d'une matrice de confusion et en utilisant le score F1, qui équilibre à la fois **la précision** et **le recall**.
- Nous avons utilisé la validation croisée et la recherche en grille pour trouver le meilleur score F1 en testant une gamme de nombres représentant des voisins. **La validation croisée** permet d'évaluer à quel point un modèle se généralise à des données non vues en divisant le jeu de données en plusieurs sous-ensembles. **La recherche en grille** permet l'optimisation des hyperparamètres en testant systématiquement des combinaisons ou des fourchettes de différentes valeurs pour les hyperparamètres, tels que le nombre de voisins.

Nous avons effectué manuellement une recherche en grille sur les effets de la régularisation en testant l'hyperparamètre `C`. Rappelez-vous, plus `C` est grand, moins il y a de régularisation. Enfin, nous avons exploré la régularisation ridge sur la régression polynomiale. Nous avons vu comment une fonction polynomiale de degré 15 peut surajuster la _ground truth_, mais après avoir appliqué une technique de régularisation (ridge ou lasso), il y a un meilleur ajustement.
